# CatBoost + XGBoost Ensemble with Multi-Target Normalization (exp_060)

**Hypothesis**: A fundamentally different approach (CatBoost + XGBoost) with multi-target normalization may have a different CV-LB relationship.

**Rationale**:
- The CV-LB intercept (0.0533) > target (0.0347) - target is mathematically unreachable with current approach
- ALL model types (MLP, LGBM, XGB, GP, Ridge, CatBoost) fall on the SAME line
- Top public kernels ("Ens Model") use CatBoost + XGBoost with different weights for single vs full data
- Multi-target normalization enforces physical constraints (yields sum ≤ 1)

**Implementation**:
1. Use CatBoost + XGBoost ensemble (NOT GP + MLP + LGBM)
2. Apply multi-target normalization: clip to [0,1], renormalize if sum > 1
3. Use different weights for single solvent (7:6) vs full data (1:2) from "Ens Model" kernel
4. Same features: Spange + DRFP + ACS PCA

**Best baseline**: exp_032 CV 0.008194 (GP 0.15 + MLP 0.55 + LGBM 0.30)

In [1]:
# Standard imports
import numpy as np
import pandas as pd
import torch
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
import tqdm
import warnings
warnings.filterwarnings('ignore')

np.random.seed(42)
torch.manual_seed(42)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')

# Check if CatBoost is available
try:
    from catboost import CatBoostRegressor
    print('CatBoost available')
except ImportError:
    print('CatBoost not available, installing...')
    import subprocess
    subprocess.run(['pip', 'install', 'catboost', '-q'])
    from catboost import CatBoostRegressor
    print('CatBoost installed')

Using device: cuda
CatBoost available


In [2]:
# Data loading functions
DATA_PATH = '/home/data'

INPUT_LABELS_NUMERIC = ["Residence Time", "Temperature"]
INPUT_LABELS_SINGLE_SOLVENT = ["Residence Time", "Temperature", "SOLVENT NAME"]
INPUT_LABELS_FULL_SOLVENT = ["Residence Time", "Temperature", "SOLVENT A NAME", "SOLVENT B NAME", "SolventB%"]

def load_data(name="full"):
    if name == "full":
        df = pd.read_csv(f'{DATA_PATH}/catechol_full_data_yields.csv')
        X = df[INPUT_LABELS_FULL_SOLVENT]
    else:
        df = pd.read_csv(f'{DATA_PATH}/catechol_single_solvent_yields.csv')
        X = df[INPUT_LABELS_SINGLE_SOLVENT]
    Y = df[["Product 2", "Product 3", "SM"]]
    return X, Y

def generate_leave_one_out_splits(X, Y):
    for solvent in sorted(X["SOLVENT NAME"].unique()):
        mask = X["SOLVENT NAME"] != solvent
        yield (X[mask], Y[mask]), (X[~mask], Y[~mask])

def generate_leave_one_ramp_out_splits(X, Y):
    ramps = X[["SOLVENT A NAME", "SOLVENT B NAME"]].drop_duplicates()
    for _, row in ramps.iterrows():
        mask = (X["SOLVENT A NAME"] == row["SOLVENT A NAME"]) & (X["SOLVENT B NAME"] == row["SOLVENT B NAME"])
        yield (X[~mask], Y[~mask]), (X[mask], Y[mask])

print('Data loading functions defined')

Data loading functions defined


In [3]:
# Load feature lookups
SPANGE_DF = pd.read_csv(f'{DATA_PATH}/spange_descriptors_lookup.csv', index_col=0)
DRFP_DF = pd.read_csv(f'{DATA_PATH}/drfps_catechol_lookup.csv', index_col=0)
ACS_PCA_DF = pd.read_csv(f'{DATA_PATH}/acs_pca_descriptors_lookup.csv', index_col=0)

# Filter DRFP to high-variance columns
drfp_variance = DRFP_DF.var()
nonzero_variance_cols = drfp_variance[drfp_variance > 0].index.tolist()
DRFP_FILTERED = DRFP_DF[nonzero_variance_cols]

print(f'Spange: {SPANGE_DF.shape}, DRFP filtered: {DRFP_FILTERED.shape}, ACS PCA: {ACS_PCA_DF.shape}')

Spange: (26, 13), DRFP filtered: (24, 122), ACS PCA: (24, 5)


In [4]:
# Full Featurizer - 145 features
class FullFeaturizer:
    def __init__(self, mixed=False):
        self.mixed = mixed
        self.spange_df = SPANGE_DF
        self.drfp_df = DRFP_FILTERED
        self.acs_pca_df = ACS_PCA_DF
        self.feats_dim = 2 + 3 + self.spange_df.shape[1] + self.drfp_df.shape[1] + self.acs_pca_df.shape[1]

    def featurize(self, X, flip=False):
        X_vals = X[INPUT_LABELS_NUMERIC].values.astype(np.float64)
        temp_c = X_vals[:, 1:2]
        time_m = X_vals[:, 0:1]
        temp_k = temp_c + 273.15
        inv_temp = 1000.0 / temp_k
        log_time = np.log(time_m + 1e-6)
        interaction = inv_temp * log_time
        X_kinetic = np.hstack([X_vals, inv_temp, log_time, interaction])
        
        if self.mixed:
            A_spange = self.spange_df.loc[X["SOLVENT A NAME"]].values
            B_spange = self.spange_df.loc[X["SOLVENT B NAME"]].values
            A_drfp = self.drfp_df.loc[X["SOLVENT A NAME"]].values
            B_drfp = self.drfp_df.loc[X["SOLVENT B NAME"]].values
            A_acs = self.acs_pca_df.loc[X["SOLVENT A NAME"]].values
            B_acs = self.acs_pca_df.loc[X["SOLVENT B NAME"]].values
            pct = X["SolventB%"].values.reshape(-1, 1)
            if flip:
                X_spange = B_spange * (1 - (1-pct)) + A_spange * (1-pct)
                X_drfp = B_drfp * (1 - (1-pct)) + A_drfp * (1-pct)
                X_acs = B_acs * (1 - (1-pct)) + A_acs * (1-pct)
            else:
                X_spange = A_spange * (1 - pct) + B_spange * pct
                X_drfp = A_drfp * (1 - pct) + B_drfp * pct
                X_acs = A_acs * (1 - pct) + B_acs * pct
        else:
            X_spange = self.spange_df.loc[X["SOLVENT NAME"]].values
            X_drfp = self.drfp_df.loc[X["SOLVENT NAME"]].values
            X_acs = self.acs_pca_df.loc[X["SOLVENT NAME"]].values
        
        return np.hstack([X_kinetic, X_spange, X_drfp, X_acs])

print(f'Full feature dimension: {FullFeaturizer().feats_dim}')

Full feature dimension: 145


In [5]:
# Multi-target normalization function
def normalize_predictions(preds):
    """Clip to [0,1] and renormalize if sum > 1."""
    # Clip to [0, 1]
    preds = np.clip(preds, 0, 1)
    
    # If sum > 1, normalize
    row_sums = preds.sum(axis=1, keepdims=True)
    mask = row_sums > 1
    preds = np.where(mask, preds / row_sums, preds)
    
    return preds

# Test normalization
test_preds = np.array([[0.3, 0.4, 0.5], [0.2, 0.3, 0.4], [0.1, 0.1, 0.1]])
print('Before normalization:', test_preds.sum(axis=1))
test_normalized = normalize_predictions(test_preds)
print('After normalization:', test_normalized.sum(axis=1))
print('Normalized predictions:', test_normalized)

Before normalization: [1.2 0.9 0.3]
After normalization: [1.  0.9 0.3]
Normalized predictions: [[0.25       0.33333333 0.41666667]
 [0.2        0.3        0.4       ]
 [0.1        0.1        0.1       ]]


In [6]:
# CatBoost + XGBoost Ensemble Model
class CatBoostXGBoostEnsemble:
    """Ensemble using CatBoost + XGBoost with multi-target normalization."""
    def __init__(self, data='single', catboost_weight=0.5, xgboost_weight=0.5, normalize=True):
        self.data = data
        self.mixed = (data == 'full')
        self.catboost_weight = catboost_weight
        self.xgboost_weight = xgboost_weight
        self.normalize = normalize
        
        self.featurizer = FullFeaturizer(mixed=self.mixed)
        
        # Models (3 per model type, one per target)
        self.catboost_models = None
        self.xgboost_models = None
        self.scaler = None
        
    def train_model(self, X, Y):
        # Prepare features
        X_feat = self.featurizer.featurize(X)
        Y_np = Y.values
        
        # Data augmentation for mixtures
        if self.mixed:
            X_flip = self.featurizer.featurize(X, flip=True)
            X_all = np.vstack([X_feat, X_flip])
            Y_all = np.vstack([Y_np, Y_np])
        else:
            X_all = X_feat
            Y_all = Y_np
        
        # Scale features
        self.scaler = StandardScaler()
        X_scaled = self.scaler.fit_transform(X_all)
        
        # Train CatBoost (3 models, one per target)
        self.catboost_models = []
        for i in range(3):
            model = CatBoostRegressor(
                iterations=500,
                learning_rate=0.05,
                depth=6,
                l2_leaf_reg=3,
                random_seed=42+i,
                verbose=False
            )
            model.fit(X_scaled, Y_all[:, i])
            self.catboost_models.append(model)
        
        # Train XGBoost (3 models, one per target)
        self.xgboost_models = []
        for i in range(3):
            model = xgb.XGBRegressor(
                n_estimators=500,
                learning_rate=0.05,
                max_depth=6,
                reg_alpha=0.1,
                reg_lambda=1.0,
                random_state=42+i,
                verbosity=0
            )
            model.fit(X_scaled, Y_all[:, i])
            self.xgboost_models.append(model)
        
    def predict(self, X):
        # Prepare features
        X_feat = self.featurizer.featurize(X)
        
        if self.mixed:
            X_flip = self.featurizer.featurize(X, flip=True)
        
        X_scaled = self.scaler.transform(X_feat)
        
        if self.mixed:
            X_flip_scaled = self.scaler.transform(X_flip)
        
        # CatBoost predictions
        catboost_preds = np.column_stack([m.predict(X_scaled) for m in self.catboost_models])
        if self.mixed:
            catboost_preds_flip = np.column_stack([m.predict(X_flip_scaled) for m in self.catboost_models])
            catboost_preds = (catboost_preds + catboost_preds_flip) / 2
        
        # XGBoost predictions
        xgboost_preds = np.column_stack([m.predict(X_scaled) for m in self.xgboost_models])
        if self.mixed:
            xgboost_preds_flip = np.column_stack([m.predict(X_flip_scaled) for m in self.xgboost_models])
            xgboost_preds = (xgboost_preds + xgboost_preds_flip) / 2
        
        # Weighted ensemble
        combined = self.catboost_weight * catboost_preds + self.xgboost_weight * xgboost_preds
        
        # Multi-target normalization
        if self.normalize:
            combined = normalize_predictions(combined)
        else:
            combined = np.clip(combined, 0, 1)
        
        return torch.tensor(combined)

print('CatBoostXGBoostEnsemble defined')

CatBoostXGBoostEnsemble defined


In [ ]:
# Quick test on a small subset
X_single, Y_single = load_data("single_solvent")
print(f'Single solvent data: X={X_single.shape}, Y={Y_single.shape}')

# Test on first fold only
test_solvent = sorted(X_single["SOLVENT NAME"].unique())[0]
mask = X_single["SOLVENT NAME"] != test_solvent

print(f'Testing on fold 0 (solvent: {test_solvent})...')

# Test with single solvent weights (7:6 from Ens Model kernel)
model = CatBoostXGBoostEnsemble(data='single', catboost_weight=7/13, xgboost_weight=6/13, normalize=True)
model.train_model(X_single[mask], Y_single[mask])
preds = model.predict(X_single[~mask])

actuals = Y_single[~mask].values
mse = np.mean((actuals - preds.numpy()) ** 2)
print(f'Test fold MSE: {mse:.6f}')
print(f'Predictions shape: {preds.shape}')
print(f'Predictions sum range: [{preds.sum(axis=1).min():.4f}, {preds.sum(axis=1).max():.4f}]')

In [ ]:
# Run full CV on single solvent data
print('\n=== Single Solvent CV (CatBoost + XGBoost, weights 7:6, normalized) ===')
X_single, Y_single = load_data("single_solvent")

split_generator = generate_leave_one_out_splits(X_single, Y_single)
all_predictions_single = []
all_actuals_single = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=24):
    (train_X, train_Y), (test_X, test_Y) = split
    
    # Single solvent weights: 7:6 from Ens Model kernel
    model = CatBoostXGBoostEnsemble(data='single', catboost_weight=7/13, xgboost_weight=6/13, normalize=True)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_single.append(predictions.numpy())
    all_actuals_single.append(test_Y.values)

preds_single = np.vstack(all_predictions_single)
actuals_single = np.vstack(all_actuals_single)
mse_single = np.mean((preds_single - actuals_single) ** 2)
print(f'Single Solvent MSE: {mse_single:.6f} (n={len(preds_single)})')

In [ ]:
# Run full CV on full data
print('\n=== Full Data CV (CatBoost + XGBoost, weights 1:2, normalized) ===')
X_full, Y_full = load_data("full")

split_generator = generate_leave_one_ramp_out_splits(X_full, Y_full)
all_predictions_full = []
all_actuals_full = []

for fold_idx, split in tqdm.tqdm(enumerate(split_generator), total=13):
    (train_X, train_Y), (test_X, test_Y) = split
    
    # Full data weights: 1:2 from Ens Model kernel
    model = CatBoostXGBoostEnsemble(data='full', catboost_weight=1/3, xgboost_weight=2/3, normalize=True)
    model.train_model(train_X, train_Y)
    predictions = model.predict(test_X)
    
    all_predictions_full.append(predictions.numpy())
    all_actuals_full.append(test_Y.values)

preds_full = np.vstack(all_predictions_full)
actuals_full = np.vstack(all_actuals_full)
mse_full = np.mean((preds_full - actuals_full) ** 2)
print(f'Full Data MSE: {mse_full:.6f} (n={len(preds_full)})')

In [ ]:
# Calculate overall MSE
n_single = len(preds_single)
n_full = len(preds_full)
overall_mse = (mse_single * n_single + mse_full * n_full) / (n_single + n_full)

print(f'\n=== CV SCORE SUMMARY (CatBoost + XGBoost with Multi-Target Normalization) ===')
print(f'Single Solvent MSE: {mse_single:.6f} (n={n_single})')
print(f'Full Data MSE: {mse_full:.6f} (n={n_full})')
print(f'Overall MSE: {overall_mse:.6f}')
print(f'\nBest CV (exp_032): 0.008194')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
    print(f'Predicted LB: {4.23 * overall_mse + 0.0533:.4f}')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')

In [ ]:
# Create submission from CV predictions
submission_single = []
for fold_idx, preds in enumerate(all_predictions_single):
    for row_idx, row in enumerate(preds):
        submission_single.append({
            "task": 0,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })
submission_single_solvent = pd.DataFrame(submission_single)

submission_full = []
for fold_idx, preds in enumerate(all_predictions_full):
    for row_idx, row in enumerate(preds):
        submission_full.append({
            "task": 1,
            "fold": fold_idx,
            "row": row_idx,
            "target_1": row[0],
            "target_2": row[1],
            "target_3": row[2]
        })
submission_full_data = pd.DataFrame(submission_full)

# Combine and save
submission = pd.concat([submission_single_solvent, submission_full_data])
submission = submission.reset_index()
submission.index.name = "id"
submission.to_csv("/home/submission/submission.csv", index=True)

print(f'Submission saved: {len(submission)} rows')
print(f'Single solvent: {len(submission_single_solvent)}, Full data: {len(submission_full_data)}')

In [ ]:
# Final summary
print(f'\n=== FINAL SUMMARY ===')
print(f'\nModel: CatBoost + XGBoost Ensemble with Multi-Target Normalization')
print(f'Single solvent weights: CatBoost=7/13, XGBoost=6/13')
print(f'Full data weights: CatBoost=1/3, XGBoost=2/3')
print(f'\nCV Results:')
print(f'  Single Solvent MSE: {mse_single:.6f}')
print(f'  Full Data MSE: {mse_full:.6f}')
print(f'  Overall MSE: {overall_mse:.6f}')
print(f'\nBaseline (exp_032): CV 0.008194')

if overall_mse < 0.008194:
    improvement = (0.008194 - overall_mse) / 0.008194 * 100
    print(f'\n✓ IMPROVEMENT: {improvement:.2f}% better than best CV!')
else:
    degradation = (overall_mse - 0.008194) / 0.008194 * 100
    print(f'\n✗ WORSE: {degradation:.2f}% worse than best CV')